In [1]:
import rasterio as rio
from rasterio.plot import show
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import earthpy.plot as ep
import shapely
import os, sys, fnmatch
from datetime import datetime

In [2]:
# Get the current directory
current_directory = os.getcwd()

# List all files in the directory
all_files = os.listdir(current_directory)

# Filter files that contain "STACKED_CROPPED" in their filename
filenames = [filename for filename in all_files if "STACKED_CROPPED" in filename]

# these files are automatically sorted by date acquired (filenaming conventions) from oldest
# to youngest
filenames

['LC08_L1TP_113037_20140423_20200911_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20151019_20200908_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20160514_20200907_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20180317_20200901_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20180808_20200831_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20190507_20200829_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20200306_20200822_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20211120_20211130_02_T1_TOA_STACKED_CROPPED.tif',
 'LC08_L1TP_113037_20230502_20230509_02_T1_TOA_STACKED_CROPPED.tif']

In [3]:
# Export NDVI rasters for each TOA image
init_src = rio.open(filenames[-1])
profile = init_src.profile
profile.update(count = 1)
init_src.close()

data_dict = {}

for file in tqdm(filenames):
    # Read in the red and near infrared bands
    src = rio.open(file)
    red = src.read(4)
    nir = src.read(5)
    src.close()
    
    # calculate ndvi
    ndvi = (nir - red) / (nir + red)
    data_dict[file.split('_')[3]] = ndvi

# # Uncomment this section if you need to save geotiffs of the NDVI data
#     output_raster = rio.open(
#         f"NDVI_{file.split('_')[3]}.tif",
#         "w",
#         **profile
#     )
    
#     output_raster.write(ndvi, 1)
#     output_raster.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:10<00:00,  1.18s/it]


In [4]:
# dates = [datetime.strptime(filename.split("_")[3], '%Y%m%d').strftime('%d %B %Y') for filename in filenames]
dates = [filename.split("_")[3] for filename in filenames]

# Sort dates by month, key is an argment that takes a function that takes in an element from the 
# list and returns some numerical output: the list will be sorted by that output
dates = sorted(dates, key = lambda x: (int(x[4:6]), int(x[6:])))
dates

['20200306',
 '20180317',
 '20140423',
 '20230502',
 '20190507',
 '20160514',
 '20180808',
 '20151019',
 '20211120']

In [31]:
# Basic NDVI visualization over time
from dateutil import parser
from matplotlib.animation import FuncAnimation

%matplotlib qt5
fig, ax = plt.subplots(figsize = (18,20))

# Initialize an image plot containing the first frame
ndvi_plot = ax.imshow(data_dict[dates[0]], cmap = "RdYlGn", vmin = 0.2, vmax = 0.6, interpolation = "nearest")

# turn off axes for visual clarity
ax.axis("off")

colorbar = plt.colorbar(ndvi_plot)

def update(frame):
    date = dates[frame]
    
    # load the image from the dictionary
    # set the image onto the figure
    ndvi_plot.set_data(data_dict[date])
    
    ax.set_title(f"Nagasaki Prefecture NDVI\n{parser.parse(date).strftime('%d %B %Y')}", fontsize = 24)

# Call the func animation class with the appropriate arguments. Frames indicates how many
# frames there will be IE how many frames will be passed into the `update` function, while
# repeat tells matplotlib whether to loop the animation, interval tells FuncAnimation how much time
# between frames
animation = FuncAnimation(fig, update, frames=len(dates), repeat=True, interval = 1000)
plt.subplots_adjust(
    top=0.91,
    bottom=0.02,
    left=0.008,
    right=0.992,
    hspace=0.2,
    wspace=0.2
)

# Save the animation
animation_file = f'ndvi_animation_by_month.gif'
animation.save(animation_file, writer='pillow')

plt.show()

In [13]:
import rasterstats

mun_bounds = gpd.read_file("mun_bounds_cleaned.geojson")

for date in dates:
    mun_ndvi = rasterstats.zonal_stats(
        "mun_bounds_cleaned.geojson",
        f"NDVI_{date}.tif"
    )
    
    mun_ndvi = pd.DataFrame(mun_ndvi)
    mun_ndvi = mun_ndvi.add_prefix(f"{date}_")
    
    mun_bounds = pd.concat([mun_bounds, mun_ndvi], axis = 1)

# dataframe now contains ndvi 
mun_bounds.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name:en         12 non-null     object  
 1   area            12 non-null     float64 
 2   geometry        12 non-null     geometry
 3   20200306_min    12 non-null     float64 
 4   20200306_max    12 non-null     float64 
 5   20200306_mean   12 non-null     float64 
 6   20200306_count  12 non-null     int64   
 7   20180317_min    12 non-null     float64 
 8   20180317_max    12 non-null     float64 
 9   20180317_mean   12 non-null     float64 
 10  20180317_count  12 non-null     int64   
 11  20140423_min    12 non-null     float64 
 12  20140423_max    12 non-null     float64 
 13  20140423_mean   12 non-null     float64 
 14  20140423_count  12 non-null     int64   
 15  20230502_min    12 non-null     float64 
 16  20230502_max    12 non-null     float64 
 17  20230502_m

In [43]:
# Basic NDVI visualization over time
from dateutil import parser
from matplotlib.animation import FuncAnimation

%matplotlib qt5
fig, ax = plt.subplots(figsize = (18,20))
ax.axis("off")
fontsize = 12

ims = []

from matplotlib.colors import LinearSegmentedColormap

# Create a custom gradient cmap
cmap = LinearSegmentedColormap.from_list(
    name='custom',
    colors=['red', 'yellow', 'green'])

# # Find the min and max of the data
# vmin = np.min(mun_bounds.loc[:, mun_bounds.columns.str.contains("mean")].to_numpy(), axis = (0,1))
# vmax = np.max(mun_bounds.loc[:, mun_bounds.columns.str.contains("mean")].to_numpy(), axis = (0,1))

# # Create a Normalizer object that will be used to set the cmap to values between the vmin and vmax
# norm = plt.Normalize(0.4, 0.5)

# # Adds axes with the following arguments: left bottom width height
# cax = fig.add_axes([0.9, 0.1, 0.03, 0.8])
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = 0.4, vmax = 0.7))
# # fake up the array of the scalar mappable. Urgh...
# sm._A = []
# fig.colorbar(sm, cax=cax)

# Adds axes with the following arguments: left bottom width height
cax = fig.add_axes([0.9, 0.1, 0.03, 0.8])
sm = plt.cm.ScalarMappable(cmap = "RdYlGn", norm=plt.Normalize(vmin = 0.4, vmax = 0.7))
# fake up the array of the scalar mappable. Urgh...
sm._A = []
fig.colorbar(sm, cax=cax)

# colorbar = plt.colorbar(ndvi_plot)
def update(frame):
    date = dates[frame]
    
    if len(ims) > 0:
        del ims[0]

    # This is apparently how you animate with gpd plot
    artist = mun_bounds.plot(ax = ax, column = f"{date}_mean", edgecolor = "black",
                             cmap = "RdYlGn", vmin = 0.4, vmax = 0.7)
    
    # Annotate municipality names
    mun_bounds.apply(lambda x: ax.annotate(text = x["name:en"], 
                                       xy = x["geometry"].centroid.coords[0],
                                       ha = "center"), axis = 1)
    
    # colorbar = plt.colorbar(artist)
    
    ims.append(artist)
    
    ax.set_title(f"Nagasaki Prefecture NDVI\n{parser.parse(date).strftime('%d %B %Y')}")
    
    return ims

# Call the func animation class with the appropriate arguments. Frames indicates how many
# frames there will be IE how many frames will be passed into the `update` function, while
# repeat tells matplotlib whether to loop the animation, interval tells FuncAnimation how much time
# between frames
animation = FuncAnimation(fig, update, frames=len(dates), repeat=True, interval = 1000)
plt.subplots_adjust(
    top=0.91,
    bottom=0.02,
    left=0.008,
    right=0.992,
    hspace=0.2,
    wspace=0.2
)

# Save the animation
animation_file = f'ndvi_animation_by_mun_copy.gif'
animation.save(animation_file, writer='pillow')

plt.show()

In [32]:
vmax

0.6943770588223922